# 🔹 Güncellenmiş YOLO + CNN Pipeline

## 1️⃣ Veri Seti Yapısı
- Train:
  - bump → Tümsek görselleri
  - road → Yol görselleri
- Test:
  - bump → Tümsek görselleri
  - road → Yol görselleri


## 2️⃣ Etiketleme
- Her görsel için **YOLO formatında .txt** dosyası oluştur:
  - bump = 0
  - road = 1
- Bounding box koordinatlarını normalize et (0–1 arası)
- Örn: `0 0.512 0.423 0.142 0.058`



## 3️⃣ Crop İşlemi
- YOLO bounding box tahminlerini kullan:
  - Tümsek veya yol alanlarını crop et
  - Crop sırasında padding bırak → çevre yol bilgisi kaybolmasın
- Crop edilmiş görseller CNN’e input olacak


## 4️⃣ Data Augmentation
- rotation, flip, brightness/contrast, zoom
- Hem YOLO hem CNN için uygulanabilir



## 5️⃣ Model Eğitimi
### YOLOv8
- Train: `train/bump + train/road`
- Val: `test/bump + test/road`
- Çıktı: bounding box + sınıf (tümsek veya yol)

### CNN
- YOLO’dan çıkan crop’lar CNN’e input
- CNN detaylı sınıflandırma yapar:
  - Tümsek tipi, yol üzerindeki çatlaklar vb.


## 6️⃣ Özet İş Akışı
1. Görsel → Etiketleme + Crop  
2. YOLOv8 → Nesne tespiti (bbox + sınıf)  
3. YOLO bbox → Crop → CNN → Detaylı sınıflandırma  
4. Çıktı → Sesli uyarı veya sistem geri bildirimi


----

# 1️ Adım: Veri Setini Okuma ve Sınıfları Tanımlama

In [3]:
import os
from pathlib import Path

base_dir=Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\YOLO\YOLO\Uygulamalar\Dataset\Speed_Bump\bump_detection_dataset")

train_dir = base_dir / "train"
test_dir = base_dir / "test"

classes = ["bump" , "road"]

for cls in classes:
    train_images = list((train_dir/cls).glob("*.jpg"))
    test_images = list((test_dir/cls).glob("*jpg"))
    
    print(f"{cls} -> Train: {len(train_images)}, Test: {len(test_images)}")


bump -> Train: 2982, Test: 1249
road -> Train: 2880, Test: 1192


# 2 Adım: Tümsek görsellerini crop edip YOLO etiketlerini oluşturmak.

### Burada yapacağımız işlem:

* Görselleri açmak

* Tümseğin bulunduğu alanı bbox ile işaretlemek

* Crop yapmak ve crop edilen görselleri kaydetmek

* YOLO formatında .txt dosyası oluşturmak

####  İşleme geçmeden önce şu soruya detay verelim ;
## Tümseğin bulunduğu yeri YOLO nasıl biliyor ?

* YOLO önceden eğitilmiş bir model veya senin eğiteceğin özel model üzerinden nesneleri öğreniyor. Yani model “tümseğin nerede olduğunu” önceden bilmez, onu görüntüdeki özelliklerden çıkarır.

# 🔹 YOLO ile Nesne Tespiti: Tümsek Örneği

## 1️⃣ Özellik Çıkarımı (Feature Extraction)
- YOLO’nun backbone kısmı (CSPDarknet gibi CNN) görseli işler  
- Görseldeki **kenarlar, köşeler, renk değişimleri, şekil desenleri** feature map olarak çıkarılır  

## 2️⃣ Grid Mantığı
- Görsel SxS boyutunda **ızgaralara bölünür** (örn. 20x20)  
- Her hücre, kendisine düşen alanda **nesne var mı yok mu** tahmin eder  

## 3️⃣ Bounding Box Tahmini
- Her hücre bir veya birkaç **anchor box** üretir  
- CNN feature map üzerinden:
  - **x, y** → kutu merkezi  
  - **w, h** → kutu boyutu  
  - **confidence score** → bu kutuda nesne var mı  
- Model bu değerleri **feature map aktivasyonlarına bakarak öğrenir**  

## 4️⃣ Sınıf Tahmini
- Aynı kutu ayrıca nesnenin **sınıfını tahmin eder** (softmax/sigmoid)  
- Örn: `[tümsek, yol, araç]` → en yüksek olasılık seçilir  

## 5️⃣ Non-Maximum Suppression (NMS)
- Aynı nesneye birden fazla kutu tahmin edilmişse, **en yüksek güven skoruna sahip kutu seçilir**  
- Böylece tek bbox ile nesne tespit edilir  

## 🔹 Özet
- YOLO **önceden tümsek nerede diye bilmez**  
- Model, **feature map’ten özellikleri çıkarır → bbox tahmin eder → sınıf atar**  
- Sen veri seti ile modelini eğittikçe, model “tümseğin yolda tipik olarak nasıl göründüğünü” öğrenir


----

# Veri setimizi biraz inceleyelim

## Veri Yolumuz aşağıdaki gibi ;

```python 

bump_detection_dataset/
│
├── train/
│   ├── bump/   # sadece görseller
│   └── road/   # sadece görseller
│
└── test/
    ├── bump/
    └── road/


* Şu an etiket (.txt) dosyaları yok, yani YOLOv8 bunu direkt kullanamaz.

### 🔹 Yapmamız gerekenler

##### Görselleri manuel veya otomatik olarak etiketlemek

* LabelImg, Roboflow veya benzeri araçlarla

* Her görsel için bounding box oluştur

* .txt dosyası otomatik üretilir

* Format: class_id x_center y_center width height (YOLO formatı)

##### Data.yaml dosyası oluşturmak

```bash 
train: "C:/Users/hdgn5/OneDrive/Masaüstü/YOLO/YOLO/Uygulamalar/Dataset/Speed_Bump/bump_detection_dataset/train"
val: "C:/Users/hdgn5/OneDrive/Masaüstü/YOLO/YOLO/Uygulamalar/Dataset/Speed_Bump/bump_detection_dataset/test"

nc: 2
names: ["bump", "road"]


##### YOLOv8 ile fine-tune eğitimi

```python 
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(
    data="data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    project="runs/train",
    name="bump_road_model"
)


## 💡 Özet:

* Mevcut görselleri etiketlemeden özel YOLO eğitimi yapamazsın.

* İnsan ve araç gibi sınıflar yok, sadece “bump” ve “road” olacak şekilde etiketle.

* Etiketleme tamamlandıktan sonra eğitim yaparsak, model yalnızca istediğimiz nesneleri tanıyacak.

---
# 🔹 Tüm görseller için YOLO format .txt oluşturma

## 🔹 Mantık

1. **Önceden eğitimli YOLOv8 modelini kullan**
   - Başlangıçta `yolov8n.pt` gibi COCO modelini kullanacağız
   - Bu model yol, tümsek, insan, araç gibi genel nesneleri tahmin edebilir

2. **Sadece bizim istediğimiz sınıfları al**
   - Örn: `bump` → sınıf 0, `road` → sınıf 1
   - İnsan veya araç bounding box’larını atla

3. **Her görsel için bounding box koordinatlarını al**
   - `results[0].boxes.xyxy` → xmin, ymin, xmax, ymax
   - `results[0].boxes.cls` → sınıf ID

4. **YOLO formatına çevir ve `.txt` oluştur**
   ```python
   x_center = (xmin + xmax) / 2 / w
   y_center = (ymin + ymax) / 2 / h
   box_w = (xmax - xmin) / w
   box_h = (ymax - ymin) / h


----
## Bunu manuel olarak uygulamadan yapalım

# 🔹 LabelImg - Görsel Etiketleme Aracı

LabelImg, YOLO veya Pascal VOC formatında görselleri etiketlemek için kullanılan **Python tabanlı bir GUI aracıdır**. Görsel üzerinde nesneleri işaretleyip `.txt` veya `.xml` dosyaları olarak kaydedebilir.



## 🔹 Mantığı

1. **Görsel Seçimi**
   - `Open Dir` ile etiketlenecek görsellerin bulunduğu klasörü seçersin.

2. **Nesne Etiketleme**
   - **Create RectBox** ile görsel üzerinde dikdörtgen kutu çizilir.
   - Her kutu bir sınıfa atanır (örn. bump → 0, road → 1).

3. **YOLO Formatı Kaydı**
   - `.txt` dosyası görsel ile **aynı isimle** oluşturulur:
     ```
     image001.jpg → image001.txt
     ```
   - İçerik:
     ```
     <class_id> <x_center> <y_center> <width> <height>
     ```
     - `x_center`, `y_center`, `width`, `height` → **0–1 arasında normalize edilmiş değerler**

4. **Pascal VOC Formatı (Opsiyonel)**
   - `.xml` dosyaları oluşturularak farklı formatta da kaydedilebilir.

5. **Kaydetme**
   - **Save** tuşuna basıldığında, seçilen kutular ve sınıflar `.txt` veya `.xml` dosyasına kaydedilir.



## 🔹 Kurulum (Windows için)

### 1️⃣ Python ile Kurulum
1. Python 3.10 veya 3.11 önerilir.
2. Gerekli paketleri yükle:
```bash
pip install pyqt5==5.15.9 lxml pillow sip


* GitHub’dan LabelImg’i klonla veya indir:

```python
git clone https://github.com/heartexlabs/labelImg.git
cd labelImg
python labelImg.py


## 🔹 İpuçları

* Klasör ve dosya isimlerinde Türkçe karakter kullanma, yoksa UnicodeEncodeError çıkar.

* YOLO formatı için .txt dosyaları görsellerle aynı ada sahip olmalı.

 Sınıflar için ID belirle:

0 → bump

1 → road

* Her kutu çizildikten sonra Save tuşuna bas → koordinatlar otomatik kaydedilir.

# 🔹 Test ve Train Seti Etiketleme Mantığı

YOLO veya CNN projelerinde veri seti ikiye ayrılır: **Train (Eğitim) ve Test (Değerlendirme)**.  



## 🔹 1️⃣ Train Seti

- **Amaç:** Modelin nesneleri öğrenmesi  
- **İşlem:**  
  - Tüm sınıfları (road, bump, crack vs.) etiketle  
  - Her görselin bounding box’larını oluştur  
  - YOLO formatında `.txt` dosyaları kaydet  
- **Önem:** Modelin genel özellikleri öğrenmesi için zorunlu



## 🔹 2️⃣ Test Seti

- **Amaç:** Modelin doğruluğunu ölçmek  
- **İşlem:**  
  - Sadece birkaç örnek görsel etiketle  
  - Her sınıftan küçük bir örnek seti yeterli (örn. 5–10 görsel)  
  - YOLO formatında `.txt` dosyaları kaydet  
- **Önem:** Performans kontrolü, hiperparametre doğrulama


## 🔹 Özet

| Set      | Etiketleme | Amaç                          |
|----------|------------|-------------------------------|
| Train    | Tüm sınıflar ve tüm görseller | Modeli eğitmek            |
| Test     | Küçük örnek seti | Model doğruluğunu ölçmek |

> Not: Test setini tamamen etiketlemene gerek yok; amaç kontrol ve değerlendirme yapmak.


## Bu verisetinde işlem yapmak için önce veri setinin şeklini değiştirmemiz gerekli.

### 📂 Olması Gereken YOLO Yapısı
```bash
bump_detection_dataset_yolo/
│
├── train/
│   ├── images/
│   │   ├── 001.jpg
│   │   ├── 002.jpg
│   │   └── ...
│   └── labels/
│       ├── 001.txt
│       ├── 002.txt
│       └── ...
│
└── val/
    ├── images/
    │   ├── xxx.jpg
    │   └── ...
    └── labels/
        ├── xxx.txt
        └── ...


Burada images/ klasöründe görseller olacak, labels/ klasöründe ise aynı isimde .txt dosyaları olacak.

### 📝 data.yaml dosyası

* YOLO için bir konfig dosyası oluşturuyoruz:

## ⚡ Otomatik klasör düzenleme (Python script)

YOLO formatına uygun yeni klasör oluşturacak bir script yazacağız. Bu aşamada sadece görseller taşınacak, .txt etiket dosyaları henüz boş olacak (çünkü bounding box bilgilerini sonra ekleyeceğiz).

In [1]:
import os
import shutil
from pathlib import Path

# Kaynak ve hedef yollar
src_root = Path(r"C:\YOLO\YOLO\Uygulamalar\Dataset\Speed_Bump_Dataset\bump_detection_dataset")
dst_root = Path(r"C:\YOLO\YOLO\Uygulamalar\Dataset\Speed_Bump_Dataset\bump_detection_dataset_yolo")

splits = ["train", "test"]  # train / test klasörleri
classes = ["bump", "road"]

for split in splits:
    for sub in ["images", "labels"]:
        os.makedirs(dst_root / split / sub, exist_ok=True)

    for cls in classes:
        img_dir = src_root / split / cls
        for img_file in img_dir.glob("*.*"):
            # Hedef image yolunu ayarla
            dst_img = dst_root / split / "images" / img_file.name
            shutil.copy(img_file, dst_img)

            # Aynı isimli label dosyasını oluştur (şimdilik boş)
            dst_lbl = dst_root / split / "labels" / (img_file.stem + ".txt")
            open(dst_lbl, "w").close()



### Yazdığımız script şu işlemleri yapıyor:
```python
* src_root/train/bump ve src_root/train/road klasörlerindeki tüm görselleri alıyor.

* Bunları dst_root/train/images/ klasörüne kopyalıyor.

* Aynı şekilde src_root/test/bump ve src_root/test/road içindeki görselleri de dst_root/test/images/ içine kopyalıyor.

* Her görsel için aynı isimli boş .txt label dosyası labels/ klasöründe oluşturuyor (çünkü YOLO için her image’ın karşılığı bir label dosyası olmalı).

### Script çalışınca hedef klasör:

```bash 
bump_detection_dataset_yolo/
├── train/
│   ├── images/
│   │   ├── img001.jpg
│   │   ├── img002.jpg
│   │   ├── img101.jpg
│   │   └── img102.jpg
│   └── labels/
│       ├── img001.txt
│       ├── img002.txt
│       ├── img101.txt
│       └── img102.txt


#### 📌 Buradaki .txt dosyaları şimdilik boş → çünkü henüz bounding box etiketleri oluşturmadık.

## Şimdi ise bu yolun içerisine bir .yaml dosyası yerleştireceğiz.


```bash
bump_detection_dataset_yolo/
├── train/
│   ├── images/
│   └── labels/
├── test/
│   ├── images/
│   └── labels/
└── data.yaml   ✅


### Şimdi diğer .ipynb dosyasına gidelim ve eğitimi gerçekleştirelim